In [1]:
# Give Google Drive Permission

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# Give your folder path

%cd gdrive/My Drive/YOUR FOLDER NAME

/content/gdrive/My Drive/Colab_Notebooks/Ociuz/1.Classification_of_Air_Pollution


In [3]:
# check folder content

!ls

 AirQualityPPT.pptx
'Classification of Air Pollution Levels using Supervised.docx'
 datasets
 frontend
 models
 Part1_Data_and_Model_Training.ipynb
 Part2_Test_The_Model.ipynb
 webapp


In [5]:
# IMport necessary libraries

import pandas as pd # to deal with daata
import joblib # to deal with saved model

In [6]:
# Get the reference data of cities and pollutants

reference_data = pd.read_csv('datasets/clean_category_data.csv')
cities = reference_data['City'].unique().tolist()
city = 'Ahmedabad'
dates = reference_data.loc[reference_data['City'] == city, 'Date'].unique().tolist()

In [7]:
# Available cities

reference_data.City.unique()

array(['Ahmedabad', 'Aizawl', 'Amaravati', 'Amritsar', 'Bengaluru',
       'Bhopal', 'Brajrajnagar', 'Chandigarh', 'Chennai', 'Coimbatore',
       'Delhi', 'Ernakulam', 'Gurugram', 'Guwahati', 'Hyderabad',
       'Jaipur', 'Jorapokhar', 'Kochi', 'Kolkata', 'Lucknow', 'Mumbai',
       'Patna', 'Shillong', 'Talcher', 'Thiruvananthapuram',
       'Visakhapatnam'], dtype=object)

## Choose your input

In [8]:
# Choose as city and date you want to predict

city = 'Ahmedabad'
date = '2017-08-02'

In [9]:
# Get the pollutant values of that day

data = reference_data[(reference_data['City'] == city) & (reference_data['Date'] == date)]
features = data[['PM2.5', 'PM10', 'NO', 'NO2', 'NOx', 'NH3', 'CO', 'SO2', 'O3']]
features

,PM2.5,PM10,NO,NO2,NOx,NH3,CO,SO2,O3
944,67.450578,118.127103,17.57473,28.560659,32.309123,23.483476,2.248598,14.531977,34.49143


In [10]:
# clean data
pollutants = list(features.columns)
concentrations = features.values.tolist()
flattened_concentrations = [val for sublist in concentrations for val in sublist]
pollutant_concentrations = dict(zip(pollutants, flattened_concentrations))

In [11]:
# load the trained model

model = joblib.load('models/xgboost_reg.pkl')

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [03:28:50] WARNING: /workspace/src/collective/../data/../common/error_msg.h:80: If you are loading a serialized model (like pickle in Python, RDS in R) or
configuration generated by an older version of XGBoost, please export the model by calling
`Booster.save_model` from that version first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/stable/tutorials/saving_model.html

for more details about differences between saving model and serializing.

  warnings.warn(smsg, UserWarning)


In [12]:
# make the air qulaity index (AQI) prediction

result = model.predict(features)
result

array([166.58932], dtype=float32)

In [13]:
# Categorieze the predicted level into Good, Moderate, Poor, etc

predict_air_quality_index = round(result[0], 2)

# Define the category mapping
category_mapping = {
    (0, 50): "Good",
    (51, 100): "Satisfactory",
    (101, 200): "Moderate",
    (201, 300): "Poor",
    (301, 400): "Very Poor",
    (401, float('inf')): "Severe"
}

# Find the category for the predicted value
category = None
for range_tuple, category_name in category_mapping.items():
    if range_tuple[0] <= predict_air_quality_index <= range_tuple[1]:
        category = category_name
        break

if category:
    print(f"The predicted air quality index is {predict_air_quality_index}, which is categorized as '{category}'.")
else:
    print("The predicted air quality index does not fall into any defined category.")

The predicted air quality index is 166.58999633789062, which is categorized as 'Moderate'.


In [14]:
# Get the health implecations of the pollutant values of that day

health_implications = {
        'PM2.5': {
            3: "Poor - Aggravation of existing respiratory conditions, leading to more frequent symptoms and decreased lung function. Increased risk of respiratory infections such as pneumonia and bronchitis",
            4: "Very Poor - Greater likelihood of experiencing severe respiratory symptoms, including difficulty breathing, chest tightness, and coughing. Worsening of cardiovascular conditions, potentially leading to heart attacks or strokes, particularly in vulnerable populations such as the elderly and those with pre-existing heart disease.",
            5: "Severe - Acute respiratory distress, with symptoms becoming more pronounced and widespread in the population. Increased risk of hospital admissions due to respiratory and cardiovascular issues. Exacerbation of chronic health conditions, posing a significant threat to public health.",
            6: "Hazardous - Critical respiratory and cardiovascular effects, including acute respiratory distress syndrome (ARDS) and cardiac events. Immediate and widespread health impacts, with potentially life-threatening consequences. Emergency measures may be required to protect public health, such as evacuation or sheltering in place."
        },
        'PM10': {
            3: "Poor- Increased risk of respiratory issues, including coughing, throat irritation, and shortness of breath, especially during physical activity or prolonged exposure.",
            4: "Very Poor - Aggravation of existing respiratory conditions, leading to more frequent symptoms and reduced lung function. Greater susceptibility to respiratory infections such as bronchitis and pneumonia.",
            5: "Severe - High respiratory symptoms in vulnerable individuals, including exacerbation of asthma and COPD. Increased risk of respiratory infections and other health issues, requiring medical attention for some individuals",
            6: "Hazardous - Severe respiratory distress, with symptoms becoming widespread and potentially life-threatening. Higher likelihood of hospital admissions due to respiratory issues and exacerbation of chronic health conditions. Emergency measures may be necessary to protect public health, including limiting outdoor activities and implementing air quality alerts."
        },
        'CO': {
            3: "Poor - Headaches, dizziness, and nausea can be experienced by sensitive individuals. Continued exposure may lead to more severe symptoms.",
            4: "Very Poor - Headaches, dizziness, and nausea become more common. People with heart disease may experience chest pain and reduced exercise tolerance.",
            5: "Severe - Extreme headaches, dizziness, and nausea are expected. Individuals with heart disease may experience significant aggravation of symptoms, including chest pain and potential cardiac arrhythmias.",
            6: "Hazardous - Life-threatening symptoms such as loss of consciousness, seizures, and coma may occur rapidly. Immediate medical attention is required, and exposure to CO at this level can be fatal without prompt treatment."
        },
        'NO2': {
            3: "Poor - Increased respiratory symptoms may occur in sensitive individuals, including worsening of asthma and other respiratory conditions. Long-term exposure may lead to decreased lung function in susceptible populations.",
            4: "Very Poor - Increased respiratory symptoms such as coughing, wheezing, and shortness of breath may occur in the general population. Individuals with pre-existing respiratory or cardiovascular conditions may experience exacerbation of symptoms.",
            5: "Severe - Significant respiratory symptoms, including severe coughing, wheezing, and shortness of breath, are expected. Risk of respiratory infections may increase due to reduced lung function and compromised immune response.",
            6: "Hazardous - Individuals may experience chest pain, palpitations, and other cardiovascular symptoms. Long-term exposure to levels this high can have serious health consequences, including permanent lung damage and increased risk of cardiovascular events. Immediate medical attention is required."
        },
        'SO2': {
            3: "Poor - Increased respiratory symptoms may occur, particularly in sensitive individuals, including worsening of asthma, increased frequency of coughing, chest tightness, and difficulty breathing.",
            4: "Very Poor - Respiratory symptoms may become more pronounced, affecting both sensitive and healthy individuals, leading to severe coughing, wheezing, shortness of breath, and chest discomfort.",
            5: "Severe - Individuals may experience significant respiratory distress, with symptoms such as severe coughing fits, difficulty breathing even at rest, wheezing audible without a stethoscope, and tightness in the chest.",
            6: "Hazardous - Severe respiratory symptoms may occur, leading to life-threatening conditions such as acute respiratory distress syndrome (ARDS), respiratory failure, severe bronchospasm, suffocation, and death, especially in vulnerable populations such as children, the elderly, and individuals with pre-existing respiratory conditions."
        },
        'O3': {
            3: "Moderate - Respiratory symptoms may worsen, particularly in sensitive individuals. This can include increased asthma symptoms, more frequent coughing, chest tightness, and slight difficulty in breathing.",
            4: "Poor - Both sensitive and healthy individuals may experience pronounced respiratory symptoms. These can include severe coughing, wheezing, moderate difficulty breathing, and discomfort in the chest.",
            5: "Very Poor - Significant respiratory distress may occur, characterized by severe coughing fits, difficulty breathing even at rest, audible wheezing, and noticeable chest tightness.",
            6: "Severe - Severe respiratory symptoms pose life-threatening risks, especially to vulnerable populations like children, the elderly, and those with existing respiratory conditions. Conditions such as acute respiratory distress syndrome (ARDS), respiratory failure, and suffocation may occur."
        }
    }

In [15]:
pollutant_health_implications = {}
for pollutant, categories in health_implications.items():
  pollutant_category = data.get(f"{pollutant}_category")
  if pollutant_category is not None:
    pollutant_health_implications[pollutant] = categories.get(pollutant_category.iloc[0], "")  # Get health implications if category is 3 to 6


pollutant_health_implications

{'PM2.5': 'Poor - Aggravation of existing respiratory conditions, leading to more frequent symptoms and decreased lung function. Increased risk of respiratory infections such as pneumonia and bronchitis',
 'PM10': 'Poor- Increased risk of respiratory issues, including coughing, throat irritation, and shortness of breath, especially during physical activity or prolonged exposure.',
 'CO': 'Poor - Headaches, dizziness, and nausea can be experienced by sensitive individuals. Continued exposure may lead to more severe symptoms.',
 'NO2': '',
 'SO2': '',
 'O3': ''}